In [ ]:
# in this file we load collected
%load_ext autoreload
%autoreload 2

import os, sys, mne, json, h5py, pymatreader, re, time, gc;
import globvars as gv
import utils
import utils_tSNE as utsne
import utils_preproc as upre
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing as mpr
import matplotlib as mpl
import scipy.signal as sig
import pandas as pd 
import utils_postprocess_HPC as postp

data_dir = gv.data_dir
from os.path import join as pjoin

from dateutil import parser
from utils_postprocess_HPC import collectCalcResults

from utils_postprocess_HPC import getMocFromRow, getMocFromRowMultiOPR

from pprint import pprint as ppr

In [ ]:
subdir = 'for_Alfons' # for output

subdir_res = 'collected'
temp_dir = pjoin( os.path.expandvars('$SCRATCH') , 'OSCBAGDIS', subdir_res )
#pjoin(gv.data_dir_tmp, 'collected')

read_saved = True
save       = False

#pptypes_to_load = ['covmat_entire', 'covmat_rest']
#pptypes_to_load = [ 'covmat_rest']
pptypes_to_load = [ 'SSS_covmat_rest']
#pptypes_to_load = [ 'covmat_rest', 'SSS_covmat_rest', 'covmat_entire']

####################
import globvars as gv
if read_saved:
    pptype2res = {}
    #df = pd.read_pickle(fnf)
    
    for ppt in pptypes_to_load:
        fn = f'{ppt}_output_per_raw.npz'
        fnf = pjoin(temp_dir, fn)
        #print(fnf)
        f = np.load(fnf, allow_pickle=1)
        pptype2res[ppt] = f['arr_0'][()]
        del f
    import gc; gc.collect()
elif save:
    df.to_pickle(fnf)
################

In [ ]:
# we use loaded data so it should be fast
use_tmpdir_to_load = True
perf_to_use_list = [('XGB','all_present_features', f'best_PCA-derived_features_0.99')]
to_show = []
to_show += [('allsep','merge_nothing','basic') ]
from utils_postprocess import  multiLevelDict2TupleList

dfs = []
for pptype,opr in pptype2res.items():
    outputs_grouped_tpll = multiLevelDict2TupleList(opr,4,3)
    outputs_grouped_tpll[0][:-1]
    dfunproc = pd.DataFrame(outputs_grouped_tpll, 
        columns = ['rawname','prefix','grouping','interval_set','mcf'] )
    prefixes = list( dfunproc['prefix'].unique() )

    # warnings.simplefilter('error')
    dftmp, table_info_per_perf_type, table_per_perf_type = \
        postp.prepTableInfo3(opr, prefixes=prefixes, 
        perf_to_use_list=perf_to_use_list, to_show=to_show, return_df = True,
        use_tmpdir_to_load = use_tmpdir_to_load)
    gc.collect()
    dftmp['pptype'] = pptype
    
    dfs += [dftmp]

df_noext = pd.concat(dfs, ignore_index=True)
import gc; gc.collect()

### load directly

In [ ]:
prefs = df_noext.query("prefix.str.contains('onlyH_act_su') | prefix.str.contains('onlyH_act_modLFP_su')")['prefix'].unique()
prefixes_to_collect = prefs

In [ ]:
from utils_postprocess_HPC import listComputedData
start_time = parser.parse("30 Oct 2019 21:21:45")  
end_time = parser.parse("30 Oct 2049 21:21:45")  
subdir_cur = 'per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_covmat_entire'
r = listComputedData(subdir_cur,None,start_time, end_time, use_main_LFP_chan=1)
r

# prefixes_to_collect = ['onlyH_act_modLFP_subskip8@both-copy_from_search_LFP',
#   'onlyH_act_modLFP_subskip8@left_exCB-copy_from_search_LFP',
#   'onlyH_act_modLFP_subskip8@right_exCB-copy_from_search_LFP',,
#   'onlyH_act_subskip8@both-copy_from_search_LFP',
#   'onlyH_act_subskip8@both_onlyCB-copy_from_search_LFP',
#   'onlyH_act_subskip8@left_exCB-copy_from_search_LFP',
#   'onlyH_act_subskip8@left_onlyCB-copy_from_search_LFP',
#   'onlyH_act_subskip8@right_exCB-copy_from_search_LFP',
#   'onlyH_act_subskip8@right_onlyCB-copy_from_search_LFP',
#   'onlyH_act_subskip8both-copy_from_search_LFP',
#   'onlyH_act_subskip8both_onlyCB-copy_from_search_LFP',
#   'onlyH_act_subskip8left_exCB-copy_from_search_LFP',
#   'onlyH_act_subskip8left_onlyCB-copy_from_search_LFP',
#   'onlyH_act_subskip8right_exCB-copy_from_search_LFP',
#   'onlyH_act_subskip8right_onlyCB-copy_from_search_LFP']

pairs = []
#pairs += [('covmat_entire','per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_covmat_entire')]
pairs += [('covmat_rest',  'per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_covmat_rest')]
pairs += [('SSS_covmat_rest',  'per_subj_per_medcond_best_LFP_wholectx_oversample_LFP256_SSS_covmat_rest')]

use_tmpdir_to_load


#prefixes_to_collect   = ['']
groupings_to_collect  = ['merge_nothing']
isets_to_collect      = ['basic']

use_tmpdir_to_load = 1
pptype2res = {}
for pptype,subdir_cur in pairs:
    print(pptype,subdir_cur)
    
    sources_type = 'parcel_aal'  # or ''
    #groupings_to_collect = ['merge_nothing']; interval_sets_to_collect = ['basic']
    groupings_to_collect = None; isets_to_collect = None
    prefixes_to_collect = None # = prefixes
    r = postp.collectPerformanceInfo3(None,prefixes_to_collect,
            interval_groupings=groupings_to_collect,
            interval_sets =  isets_to_collect,
            nraws_used='[0-9]+', sources_type = sources_type,
            printFilenames=1, ndays_before=None,
            use_main_LFP_chan=1, remove_large_items = 1,                
            list_only=0, allow_multi_fn_same_prefix=0,
            use_light_files = 1, rawname_regex_full=0,
            subdir=subdir_cur,
            start_time=start_time,
            end_time=end_time, load=1, use_tmpdir_to_load=use_tmpdir_to_load)
    
    pptype2res[pptype]  = output_per_raw_cur

In [ ]:
import gc; gc.collect()

# print tables

In [ ]:
d = {'covmat_entire': pptype2res['covmat_entire']}
#%debug
dftmp = extendDfMultiOPR(df_noext,d)

In [ ]:
#from utils_postprocess_HPC import extendDF
from utils_postprocess_HPC import extendDfMultiOPR
#%debug
df2 = extendDfMultiOPR(df_noext,pptype2res)
df = df2

In [ ]:
df_all = df

In [ ]:
from globvars import gp
cols_pctpts = []
for it in gp.int_types_basic:
    cols_pctpts += [f'numpts_{it}']
cols_pctpts

In [ ]:
#prefix_cur_table0 = 'onlyH_act_modLFP'
prefix_cur_table0 = 'onlyH_act'
prefix_cur_table1 = 'onlyH_mob'
prefix_cur_table2 = 'onlyH_compl'
prefix_cur_table = 'onlyH'
prefl = [f'{prefix_cur_table0}_subskip8@B-B',
         f'{prefix_cur_table1}_subskip8@B-B',
         f'{prefix_cur_table2}_subskip8@B-B',
         f'{prefix_cur_table}_subskip8@B-B']
df_all_sub = df_all.query(f'prefix_templ in {prefl} and '
    'grouping == "merge_nothing" and interval_set == "basic"')#.copy()

#'rawname'
srt = df_all_sub.sort_values(by=['medcond','subject','prefix_templ','pptype'])
r = srt[['pptype','prefix_templ', 'subject', 'medcond','bacc']  + \
      [ 'mainLFPside', 'mainLFPside_templ']].copy()
#'numpts',
r['prefix_templ'] = r['prefix_templ'].str.replace('_subskip8@B-B','')
r['prefix_templ'] = r['prefix_templ'].replace({'onlyH':'all 3 Hjorth params'})
display(r)
rr = r.groupby(['prefix_templ','medcond'])
#rr = r.groupby(['prefix_templ'])
print('mean = ') 
display( rr.mean(numeric_only=1) )
print('std = ')
display( rr.std(numeric_only=1) )

In [ ]:
#prefix_cur_table = 'onlyH_act_modLFP'
#prefl = [f'{prefix_cur_table}_subskip8@B-B']
df_all_sub = df_all.query(f'grouping == "merge_nothing" and interval_set == "basic" and rawname == "S01_off"'
                         ' and prefix.str.contains("LFPand") and medcond == "off"').copy()

#'rawname'
srt = df_all_sub.sort_values(by=['parcel_group_names','pptype'])
r = srt[['pptype','prefix_templ', 'subject', 'medcond','bacc']  + \
      ['numpts2', 'num', 'mainLFPside', 'mainLFPside_templ', 'parcel_group_names']]

display(r)
# rr = r.groupby(['prefix_templ','medcond'])
# #rr = r.groupby(['prefix_templ'])
# print('mean = ') 
# display( rr.mean(numeric_only=1) )
# print('std = ')
# display( rr.std(numeric_only=1) )